The goal here is to simulate a realistic GPS signal, including transmitter and receiver clock errors, as well as signal propagation delay based on satellite and receiver positions. The code snippets below show how to set up the simulation parameters, retrieve satellite positions from SP3 data, and compute the necessary delays and clock errors.

We will not include relativistic effects or atmospheric delays in this simplified model.

In [ ]:
import os
import utils
from datetime import datetime, timedelta
import gnss_tools.orbits as orbits
import gnss_tools.time as time
import gnss_tools.coords as coords
import numpy as np
import matplotlib.pyplot as plt

SPEED_OF_LIGHT = 299792458.0  # m/s

In [ ]:
# Create local-data directory for storing downloaded data files
data_dir = os.path.join(os.path.dirname(os.path.dirname(utils.__file__)), "local-data")
os.makedirs(data_dir, exist_ok=True)

Simulate GNSS signal assuming synchronized clocks

In [ ]:
# Define simulation time period
sim_start_dt = datetime(2024, 5, 10)
sim_end_dt = sim_start_dt + timedelta(days=1)
sim_start_gpst = time.convert_datetime_to_gps_seconds(sim_start_dt)
sim_end_gpst = time.convert_datetime_to_gps_seconds(sim_end_dt)

# Define static RX position
rx_pos_geo = np.array((-118.57707656024262, 34.139012915514925, 361.0))  # lon, lat, alt in meters
rx_pos_ecf = coords.geo2ecf(rx_pos_geo)

# Download and parse SP3 orbit data
sp3_arrays = orbits.download_and_parse_sp3_data(sim_start_gpst, sim_end_gpst, data_dir, True)

sat_positions_ecf = {}
sat_positions_sky = {}
propagation_delays = {}
geom_sim_epochs = np.arange(sim_start_gpst, sim_end_gpst, 1.0)  # every 1 seconds
for sat_id in sp3_arrays.position.keys():
    sat_pos_ecf = sp3_arrays.position[sat_id]
    sp3_epochs = sp3_arrays.epochs
    lagrange_order = 7
    sat_positions_ecf[sat_id] = orbits.compute_array_lagrange_interpolation(
        geom_sim_epochs - sim_start_gpst, sp3_epochs - sim_start_gpst, sat_pos_ecf, lagrange_order
    )
    sat_positions_sky[sat_id] = coords.ecf2sky(rx_pos_ecf, sat_positions_ecf[sat_id])
    geometric_range = np.linalg.norm(sat_positions_ecf[sat_id] - rx_pos_ecf, axis=1)
    propagation_delays[sat_id] = geometric_range / SPEED_OF_LIGHT

In [ ]:
# Recompute propagation delays to account for Earth rotation during signal travel time
corrected_propagation_delays = {}
for sat_id in sat_positions_ecf.keys():
    sat_pos_ecf = sat_positions_ecf[sat_id]
    prop_delay = propagation_delays[sat_id]
    sat_pos_rotated = coords.rotate_pos_ecf(sat_pos_ecf, -prop_delay)
    geometric_range = np.linalg.norm(sat_pos_rotated - rx_pos_ecf[None, :], axis=1)
    corrected_propagation_delays[sat_id] = geometric_range / SPEED_OF_LIGHT

In [ ]:
# Make sky plot of all satellites

# Check for elevation > 0 deg
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection="polar")
ax.set_theta_direction(-1)
ax.set_theta_offset(np.pi / 2.0)
gnss_colors = {
    "G": "blue",
    "R": "green",
    "B": "red",
    "E": "orange",
}
for sat_id, sat_pos_sky in sat_positions_sky.items():
    elevation = sat_pos_sky[:, 1]
    azimuth = sat_pos_sky[:, 0]
    mask = elevation > 0.0
    svid_sys = sat_id[0]
    if svid_sys not in gnss_colors:
        continue
    color = gnss_colors[svid_sys]
    ax.scatter(
        np.radians(azimuth[mask]),
        90.0 - elevation[mask],
        color=color,
        s=10,
    )
rticks = [0, 20, 40, 60, 80]
ax.set_yticks(rticks)
ax.set_yticklabels([str(90 - rtick) for rtick in rticks])
labels = list(gnss_colors.keys())
handles = [plt.Line2D([0], [0], lw=4, color=gnss_colors[label]) for label in labels]
ax.legend(handles, labels, loc="upper right", title="GNSS System")
plt.show()

In [ ]:
# Plot propagation delays for each satellite (when elevation > 0 deg)
fig, ax = plt.subplots(figsize=(10, 6))
plot_times = (geom_sim_epochs - sim_start_gpst) / 3600.0  # in hours
for sat_id in sat_positions_ecf.keys():
    sat_pos_sky = sat_positions_sky[sat_id]
    elevation = sat_pos_sky[:, 1]
    mask = elevation > 0.0
    ax.plot(
        plot_times[mask],
        corrected_propagation_delays[sat_id][mask] * 1e3,
        label=sat_id,
    )
ax.set_xlabel("GPS Time [hours]")

In [ ]:
samp_rate = 5e6  # 5 MHz
sim_duration = 1.0  # seconds
t = np.arange(0, sim_duration, 1 / samp_rate)